In [1]:
import pickle
import keras
from gensim.models import Word2Vec
import numpy as np

2024-03-12 20:36:33.791900: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

model = keras.models.load_model('model.h5')

word2vec_model = Word2Vec.load("word2vec_model.bin")

In [3]:
def text_to_embeddings(text, word2vec_model):
    tokens = tokenizer(text)
    embeddings = []
    for token in tokens:
        if token in word2vec_model.wv:
            embeddings.append(word2vec_model.wv[token])
    if embeddings:
        return np.mean(embeddings, axis=0) 
    else:
        return np.zeros(50)


In [4]:
tuple_test_data = (
    ('zucker fabrik', 'ft'),
    ('Lebensmittel kommssionierung', 'ft'),
    ('geländer biegen', 'mr'),
    ('gebäudeausrüstung technische', 'ct'),
    ('kürbiskernöl softgels', 'ft')
)

In [7]:
def predict_word(word: str):
    new_word_embedding = text_to_embeddings(word, word2vec_model).reshape(1, -1) 
    # Predict the label for the embedded word
    predicted_probabilities = model.predict(new_word_embedding)
    predicted_label_index = predicted_probabilities.argmax(axis=1)[0]
    predicted_label = label_encoder.inverse_transform([predicted_label_index])
    return predicted_label[0]

predict_word(word='Lebensmittel kommssionierung')

1/1 [==============================] - 0s 26ms/step


'ft'

In [5]:
for sample in tuple_test_data:
    word = sample[0]
    correct = sample[1]
    new_word_embedding = text_to_embeddings(word, word2vec_model).reshape(1, -1) 

    # Predict the label for the embedded word
    predicted_probabilities = model.predict(new_word_embedding)
    predicted_label_index = predicted_probabilities.argmax(axis=1)[0]
    predicted_label = label_encoder.inverse_transform([predicted_label_index])

    print(f'For word \'{word}\'')
    print(f'We predict {predicted_label[0]}')
    print(f'The correct solution is {correct}')
    print()


1/1 [==============================] - 0s 236ms/step
For word 'zucker fabrik'
We predict ct
The correct solution is ft

1/1 [==============================] - 0s 31ms/step
For word 'Lebensmittel kommssionierung'
We predict ft
The correct solution is ft

1/1 [==============================] - 0s 31ms/step
For word 'geländer biegen'
We predict mr
The correct solution is mr

1/1 [==============================] - 0s 21ms/step
For word 'gebäudeausrüstung technische'
We predict ch
The correct solution is ct

1/1 [==============================] - 0s 21ms/step
For word 'kürbiskernöl softgels'
We predict ch
The correct solution is ft

